In [2]:
%pip install biopython

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install datasets


  Using cached huggingface_hub-0.34.4-py3-none-any.whl.metadata (14 kB)
Using cached huggingface_hub-0.34.4-py3-none-any.whl (561 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 3.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2━━━━━━━━━━━━━━━━━━ 3/6 [fsspec]
    Uninstalling fsspec-2025.3.2:╺━━━━━━━━━━━━━━━━━━━ 3/6 [fsspec]
      Successfully uninstalled fsspec-2025.3.2━━━━━━━━━━━━━━━━ 3/6 [fsspec]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [datasets]5/6 [datasets]ce-hub]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2025.3.2 requires fsspec==2025.3.2.*, but you have fsspec 2025.3.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [9]:
# Install required packages if not already installed
# !pip install transformers torch datasets numpy

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
import numpy as np
import os
import random


In [10]:


from torch.utils.data import Dataset
from Bio import SeqIO

class FASTADataset(Dataset):
    def __init__(self, fasta_file, tokenizer, max_length=256):
        # Load sequences from a FASTA file
        self.sequences = [str(record.seq) for record in SeqIO.parse(fasta_file, "fasta")]
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        # Return number of sequences
        return len(self.sequences)

    def __getitem__(self, idx):
        # Fetch one sequence and tokenize it
        seq = self.sequences[idx]
        encoding = self.tokenizer(
            seq,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {key: val.squeeze(0) for key, val in encoding.items()}


In [11]:
import torch
import random
import os

# --- CHANGE THIS PATH to your folder containing .pt files ---
data_folder = r"C:\Users\rajpu\Desktop\SEM5\AFML\PROJECT\fnet_train_data_256"

train_path = os.path.join(data_folder, "train_dataset.pt")
val_path   = os.path.join(data_folder, "val_dataset.pt")
test_path  = os.path.join(data_folder, "test_dataset.pt")

# Function to load only the first N items from a .pt file
def load_partial_pt(path, n):
    all_data = torch.load(path, weights_only=False)
    return list(all_data)[:n]

# Set exact number of sequences
N_TRAIN = 4000
N_VAL   = 500
N_TEST  = 500

random.seed(42)  # for reproducibility

# Load only the subset needed
train_sequences = load_partial_pt(train_path, N_TRAIN)
val_sequences   = load_partial_pt(val_path, N_VAL)
test_sequences  = load_partial_pt(test_path, N_TEST)

print("Subset sizes:")
print(f" Training sequences: {len(train_sequences)}")
print(f" Validation sequences: {len(val_sequences)}")
print(f" Test sequences: {len(test_sequences)}")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\rajpu\\Desktop\\SEM5\\AFML\\PROJECT\\fnet_train_data_256/train_dataset.pt'

In [29]:
# --- CHANGE MODEL NAME if needed ---
tokenizer = AutoTokenizer.from_pretrained("google/fnet-base")

# Load FNet for Masked Language Modeling
model = AutoModelForMaskedLM.from_pretrained("google/fnet-base")

print("Tokenizer and model loaded successfully.")


Tokenizer and model loaded successfully.


In [31]:
class KinaseTensorDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        # Each item is already a tensor dictionary: input_ids, attention_mask
        return {k: v.clone() for k, v in self.sequences[idx].items()}

# Create datasets
train_dataset = KinaseTensorDataset(train_sequences)
val_dataset   = KinaseTensorDataset(val_sequences)
test_dataset  = KinaseTensorDataset(test_sequences)

print("Datasets ready.")


Datasets ready.


In [32]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)


In [8]:
# check your current transformers version
!python -c "import transformers; print('transformers version:', transformers.__version__)"

# upgrade to latest transformers, datasets, and accelerate
%pip install --upgrade transformers datasets accelerate

# (optional but often useful for training)
%pip install --upgrade evaluate


Traceback (most recent call last):
  File "<string>", line 1, in <module>
    import transformers; print('transformers version:', transformers.__version__)
    ^^^^^^^^^^^^^^^^^^^
ModuleNotFoundError: No module named 'transformers'
  Using cached transformers-4.56.1-py3-none-any.whl.metadata (42 kB)
  Using cached tokenizers-0.22.0-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached safetensors-0.6.2-cp38-abi3-macosx_11_0_arm64.whl.metadata (4.1 kB)
Using cached transformers-4.56.1-py3-none-any.whl (11.6 MB)
Using cached tokenizers-0.22.0-cp39-abi3-macosx_11_0_arm64.whl (2.9 MB)
Using cached safetensors-0.6.2-cp38-abi3-macosx_11_0_arm64.whl (432 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [transformers] [transformers]
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [18]:
import transformers
print(transformers.__version__)



4.56.1


In [33]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./KinaseFNet",
    overwrite_output_dir=True,
    num_train_epochs=20,                
    per_device_train_batch_size=8,      
    per_device_eval_batch_size=8,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=False  # disable half precision to fix cuFFT error
)


In [1]:
import evaluate

accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    mask = labels != -100
    preds = np.argmax(logits, axis=-1)
    preds = preds[mask]
    labels = labels[mask]
    return accuracy_metric.compute(predictions=preds, references=labels)

model = model.to(torch.float32)

ModuleNotFoundError: No module named 'evaluate'

In [35]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()


C:\Users\rajpu\AppData\Local\Temp\ipykernel_18516\1933361019.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': None, 'bos_token_id': None}.


RuntimeError: cuFFT only supports dimensions whose sizes are powers of two when computing in half precision, but got a signal size of[256, 768]

In [ ]:
# Evaluate on validation set
val_results = trainer.evaluate(val_dataset)
print("Validation set results:", val_results)

# Evaluate on test set
test_results = trainer.evaluate(test_dataset)
print("Test set results:", test_results)


In [ ]:
trainer.save_model("./KinaseFNet")
tokenizer.save_pretrained("./KinaseFNet")
print("KinaseFNet fine-tuned model saved.")


In [ ]:
test_results = trainer.evaluate(test_dataset)
print("Test set results:", test_results)
